In [ ]:
!git clone https://github.com/Bhavnoor-Coders-1010/food101.git

fatal: destination path 'food101' already exists and is not an empty directory.


In [ ]:
%cd food101

/content/food101


In [ ]:
%ls models

model_epoch_01.keras  model_epoch_02.keras  model_epoch_09.keras


In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers,models
import matplotlib.pyplot as plt
import os

In [ ]:
df = pd.read_csv('DataFOOD101/train.csv')
train_df=df.copy()
df_2 = pd.read_csv('DataFOOD101/test.csv')
test_df=df_2.copy()

print(train_df.head())

print(test_df.head())


image_folder_train = 'DataFOOD101/Train/'
image_folder_test = 'DataFOOD101/Test/'

class_names = {
  0 : "burger",
  1 : "butter_naan",
  2 : "chai",
  3 : "chapati",
  4 : "chole_bhature",
  5 : "dal_makhani",
  6 : "dhokla",
  7 : "fried_rice",
  8 : "idli",
  9 : "jalebi",
  10 : "kadhai_paneer",
  11 : "kathi_roll",
  12 : "kulfi",
  13 : "masala_dosa",
  14 : "momos",
  15 : "paani_puri",
  16 : "pakode",
  17 : "pav_bhaji",
  18 : "pizza",
  19 : "samosa"
}

for food in class_names.values():
  os.makedirs(os.path.join(image_folder_train, food), exist_ok = True)
  os.makedirs(os.path.join(image_folder_test, food), exist_ok = True)

for i, row in train_df.iterrows():
  image_path = os.path.join(image_folder_train + row["imagePath"])
  new_path = os.path.join(image_folder_train, f"{class_names[row['label']]}/" + row["imagePath"])
  if(os.path.isfile(image_path)):
    os.replace(image_path, new_path)

for i, row in test_df.iterrows():
  image_path = os.path.join(image_folder_test + row["imagePath"])
  new_path = os.path.join(image_folder_test, f"{class_names[row['label']]}/" + row["imagePath"])
  if(os.path.isfile(image_path)):
    os.replace(image_path, new_path)

  imagePath  label
0  0342.jpg      2
1  1077.jpg      0
2  2314.jpg     11
3  3068.jpg      0
4  4121.jpg     13
   Unnamed: 0 imagePath  label
0           0  0205.jpg      5
1           1  1134.jpg      3
2           2  2231.jpg      5
3           3  3401.jpg      4
4           4  4280.jpg     10


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size = 32
IMG_SIZE = (224, 224)

train_datagen = ImageDataGenerator(rescale=1. / 255,
                                   validation_split=0.2,
                                   rotation_range=30,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(image_folder_train,
                                                           target_size = IMG_SIZE,
                                                           class_mode='sparse',
                                                           batch_size = batch_size,
                                                           subset='training',
                                                           shuffle=True)

validation_generator = train_datagen.flow_from_directory(image_folder_train,
                                                           target_size = IMG_SIZE,
                                                           class_mode='sparse',
                                                           batch_size = batch_size,
                                                           subset='validation',
                                                           shuffle=True)
# test_data_augmentor = ImageDataGenerator(rescale = 1. / 255)
# test_generator = test_data_augmentor.flow_from_directory(image_folder_test,
#                                                            target_size = IMG_SIZE,
#                                                            batch_size = batch_size,
#                                                            class_mode = 'categorical')

Found 4260 images belonging to 20 classes.
Found 1054 images belonging to 20 classes.


In [ ]:
class_map = train_generator.class_indices
class_map #kadhai_paneer and kathi_roll are swapped as compared to `class_names`.

{'burger': 0,
 'butter_naan': 1,
 'chai': 2,
 'chapati': 3,
 'chole_bhature': 4,
 'dal_makhani': 5,
 'dhokla': 6,
 'fried_rice': 7,
 'idli': 8,
 'jalebi': 9,
 'kadhai_paneer': 10,
 'kathi_roll': 11,
 'kulfi': 12,
 'masala_dosa': 13,
 'momos': 14,
 'paani_puri': 15,
 'pakode': 16,
 'pav_bhaji': 17,
 'pizza': 18,
 'samosa': 19}

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=IMG_SIZE+(3,))

for layer in base_model.layers:
    layer.trainable = False

model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.BatchNormalization(),
    layers.Dense(units=128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=20, activation='softmax')
])

In [ ]:
from keras.callbacks import ModelCheckpoint

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

checkpoint_callback = ModelCheckpoint(
    filepath='models/model_epoch_{epoch:02d}.keras',
    monitor='val_accuracy',
    save_best_only=True,
    save_freq='epoch',
)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

trained_model = model.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator,
    callbacks=[checkpoint_callback]
)

Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


134/134 ━━━━━━━━━━━━━━━━━━━━ 365s 3s/step - accuracy: 0.5085 - loss: 1.8048 - val_accuracy: 0.7163 - val_loss: 1.1269
Epoch 2/25
134/134 ━━━━━━━━━━━━━━━━━━━━ 366s 3s/step - accuracy: 0.7707 - loss: 0.7598 - val_accuracy: 0.7647 - val_loss: 0.8997
Epoch 3/25
134/134 ━━━━━━━━━━━━━━━━━━━━ 362s 2s/step - accuracy: 0.8203 - loss: 0.5685 - val_accuracy: 0.7429 - val_loss: 1.1139
Epoch 4/25
134/134 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.8407 - loss: 0.5105 - val_accuracy: 0.7533 - val_loss: 1.0140
Epoch 5/25
134/134 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.8896 - loss: 0.3530 - val_accuracy: 0.7334 - val_loss: 1.1621
Epoch 6/25
134/134 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.8890 - loss: 0.3385 - val_accuracy: 0.7448 - val_loss: 1.1036
Epoch 7/25
134/134 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.9099 - loss: 0.2847 - val_accuracy: 0.7476 - val_loss: 1.1324
Epoch 8/25
134/134 ━━━━━━━━━━━━━━━━━━━━ 324s 2s/step - accuracy: 0.9144 - loss: 0.2628 - val_accuracy: 0.766

In [ ]:
untuned_model_save = model.save("models/MobileNetV2Untuned.keras")

In [ ]:
for layer in base_model.layers[-20:]:
  if not isinstance(layer, layers.BatchNormalization):
    layer.trainable = True

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.00001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

trained_model_tuned = model.fit(
    train_generator,
    epochs=7,
    validation_data=validation_generator,
    callbacks=[checkpoint_callback]
)

Epoch 1/7
134/134 ━━━━━━━━━━━━━━━━━━━━ 387s 3s/step - accuracy: 0.9560 - loss: 0.1170 - val_accuracy: 0.7713 - val_loss: 1.2876
Epoch 2/7
134/134 ━━━━━━━━━━━━━━━━━━━━ 390s 3s/step - accuracy: 0.9567 - loss: 0.1071 - val_accuracy: 0.7723 - val_loss: 1.2745
Epoch 3/7
134/134 ━━━━━━━━━━━━━━━━━━━━ 422s 3s/step - accuracy: 0.9624 - loss: 0.1205 - val_accuracy: 0.7619 - val_loss: 1.3517
Epoch 4/7
134/134 ━━━━━━━━━━━━━━━━━━━━ 395s 3s/step - accuracy: 0.9654 - loss: 0.1073 - val_accuracy: 0.7704 - val_loss: 1.4439
Epoch 5/7
134/134 ━━━━━━━━━━━━━━━━━━━━ 369s 3s/step - accuracy: 0.9667 - loss: 0.0999 - val_accuracy: 0.7723 - val_loss: 1.8273
Epoch 6/7
134/134 ━━━━━━━━━━━━━━━━━━━━ 394s 3s/step - accuracy: 0.9713 - loss: 0.0907 - val_accuracy: 0.7732 - val_loss: 2.1951
Epoch 7/7
134/134 ━━━━━━━━━━━━━━━━━━━━ 380s 3s/step - accuracy: 0.9741 - loss: 0.0682 - val_accuracy: 0.7704 - val_loss: 2.5007


In [ ]:
model.save("Food_classifier.keras")

In [ ]:
plt.plot(trained_model.history['accuracy'], label="Training accuracy")
plt.plot(trained_model.history['val_accuracy'], label="validation Accuracy")
plt.title("Model accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# Plot loss....
plt.plot(trained_model.history['loss'], label="Training Loss")
plt.plot(trained_model.history['val_loss'], label="validation Loss")
plt.title("Model Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
from keras.preprocessing import image

def predict(file_path, model):
  img = image.load_img(file_path, target_size=IMG_SIZE)
  img = image.img_to_array(img)
  img = img / 255.0
  img = np.expand_dims(axis=0)

  prediction = model.predict(img)

  predicted_class = class_map[np.argmax(prediction)]
  return predicted_class

saved_model = keras.models.load_model("Food_classifier.h5")


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_generator = test_datagen.flow_from_directory(image_folder_test,
                                                           target_size = IMG_SIZE,
                                                           class_mode='sparse',
                                                           batch_size = batch_size)

Found 939 images belonging to 20 classes.


In [ ]:
scores = saved_model.evaluate(test_generator)
print("%s%s: %.2f%%" % ("evaluate ",saved_model.metrics_names[1], scores[1]*100))


30/30 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.8150 - loss: 5.2487
evaluate compile_metrics: 79.87%


In [ ]:
import cv2
def pred_img(img_path):
  img = cv2.imread(img_path)
  resize_img = cv2.resize(img, (224,224))
  normalize_img = resize_img / 255
  img_array = np.array(normalize_img)
  img_array = np.expand_dims(img_array, axis=0)
  prediction = saved_model.predict(img_array)
  predicted_class = np.argmax(prediction,axis=1)
  return predicted_class[0]
result = pred_img("/content/dhokla.jpg")
class_names[result]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


'dhokla'

In [ ]:
model.save("Food_classifier.h5")